Установка и импорт необходимых библиотек

In [ ]:
import nltk
from nltk.corpus import stopwords
from pymorphy3 import MorphAnalyzer
from collections import defaultdict
import math
import re
# Загрузка стоп-слов для русского языка
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mg821\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Загрузка стоп-слов и инициализация инструментов

In [ ]:
stop_words = set(stopwords.words('russian'))

# Инициализация морфологического анализатора и стеммера
morph = MorphAnalyzer()

функции для обработки текста

In [69]:
def lemmatize_word(word):
    """Возвращает лемматизированную форму слова."""
    return morph.parse(word)[0].normal_form


def preprocessing(texts: list[str]) -> list[list[str]]:
    """
    Препроцессинг текстов: удаление знаков препинания, удаление стоп-слов, токенизация, лемматизация.
    """
    result = []
    for text in texts:
        # Удаление знаков препинания и цифр
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\d+', '', text)
        
        # Преобразование в нижний регистр
        text = text.lower()
        
        # Разделение на слова
        words = text.split()
        
        # Удаление стоп-слов и лемматизация
        processed_words = [lemmatize_word(word) for word in words if word not in stop_words]
        
        result.append(processed_words)
    
    return result


 Функция перевода текста с русского на английский, Токенизация текста

In [ ]:
def tokenize_text(text):
    """Токенизирует текст на уровне отдельных символов."""
    return list(text)

Реализация Bag of Words

In [71]:
def make_dict(texts: list[list[str]]) -> dict[str]:
    """
    Создание словаря, где ключами являются уникальные слова, а значениями — их индексы.
    """
    word_dict = {}
    index = 0
    
    for text in texts:
        for word in text:
            if word not in word_dict:
                word_dict[word] = index
                index += 1
    
    return word_dict

def bag_of_words(texts: list[list[str]], word_dict: dict[str]) -> list[list[int]]:
    """
    Создание Bag of Words представления текстов.
    """
    result = []
    
    for text in texts:
        bag = [0] * len(word_dict)
        for word in text:
            if word in word_dict:
                bag[word_dict[word]] += 1
        result.append(bag)
    
    return result

Реализация TF-IDF

In [72]:
def count_num_words(texts: list[list[str]]) -> list[int]:
    """
    Подсчет числа слов в каждом тексте.
    """
    return [len(text) for text in texts]

def count_word_occurrences(texts: list[list[str]], word_dict: dict[str]) -> dict[int, int]:
    """
    Подсчет количества вхождений каждого слова в текстах.
    """
    word_occurrences = defaultdict(int)
    
    for text in texts:
        for word in text:
            if word in word_dict:
                word_occurrences[word_dict[word]] += 1
    
    return dict(word_occurrences)

def tfidf(texts: list[list[str]], word_dict: dict[str]) -> list[list[float]]:
    """
    Создание TF-IDF представления текстов.
    """
    num_words = count_num_words(texts)
    word_occurrences = count_word_occurrences(texts, word_dict)
    total_texts = len(texts)
    
    result = []
    
    for text in texts:
        tf = [0] * len(word_dict)
        for word in text:
            if word in word_dict:
                index = word_dict[word]
                tf[index] = (tf[index] + 1) / num_words[texts.index(text)]
        
        idf = [math.log(total_texts / (occurrences + 1)) for occurrences in word_occurrences.values()]
        
        tf_idf = [tf[i] * idf[i] for i in range(len(word_dict))]
        
        result.append(tf_idf)
    
    return result

Вывод результатов

In [ ]:
# Исходный текст
texts = [
    "Сегодня погода отличная, я пошла на прогулку.",
    "Я очень рада, что сегодня так солнечно.",
    "Вчера было дождливо, я не могла на прогулку."
]

# Препроцессинг
preprocessed_texts = preprocessing(texts)

# Вывод результата
print("-" * 40)
print("📄 Исходные тексты:")
for i, text in enumerate(texts):
    print(f"Текст {i+1}: {text}")
print("-" * 40)

print("\n🔹 Обработанные тексты:")
for i, text in enumerate(preprocessed_texts):
    print(f"Текст {i+1}: {text}")
print("-" * 40)

# Bag of Words
word_dict = make_dict(preprocessed_texts)
bow = bag_of_words(preprocessed_texts, word_dict)
print("\n🔹 Bag of Words:")
for i, text in enumerate(bow):
    print(f"Текст {i+1}: {text}")
print("-" * 40)

# TF-IDF
tfidf_result = tfidf(preprocessed_texts, word_dict)
print("\n🔹 TF-IDF:")
for i, text in enumerate(tfidf_result):
    print(f"Текст {i+1}: {text}")
print("-" * 40)

----------------------------------------
📄 Исходные тексты:
Текст 1: Сегодня погода отличная, я пошла на прогулку.
Текст 2: Я очень рада, что сегодня так солнечно.
Текст 3: было дождливо, я не могла на прогулку.
----------------------------------------

🔹 Обработанные тексты:
Текст 1: ['сегодня', 'погода', 'отличный', 'пойти', 'прогулка']
Текст 2: ['очень', 'рада', 'сегодня', 'солнечно']
Текст 3: ['дождливый', 'мочь', 'прогулка']
----------------------------------------

🔹 Bag of Words:
Текст 1: [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
Текст 2: [1, 0, 0, 0, 0, 1, 1, 1, 0, 0]
Текст 3: [0, 0, 0, 0, 1, 0, 0, 0, 1, 1]
----------------------------------------

🔹 TF-IDF:
Текст 1: [0.0, 0.08109302162163289, 0.08109302162163289, 0.08109302162163289, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Текст 2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.1013662770270411, 0.1013662770270411, 0.1013662770270411, 0.0, 0.0]
Текст 3: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.13515503603605478, 0.13515503603605478]
-------------------------------